## Visualize

In [14]:
import h5py
import numpy as np
import os

import sys
sys.path.append('..')
from util.plotting import plot_image_rows



def get_path(model_sim, ws, encoded, norm):
    if model_sim == "p21c":
        dp = f"/users/jsolt/data/jsolt/21cmFAST_sims/p21c14/p21c14"
    elif model_sim == "zreion":
        dp = f"/users/jsolt/data/jsolt/zreion_sims/zreion24/zreion24"
    elif model_sim == "ctrpx":
        dp = f"/users/jsolt/data/jsolt/centralpix_sims/centralpix05/centralpix05"
    if encoded:
        dp += f"_norm_encoded_ws{ws}.hdf5" if norm else f"_encoded_ws{ws}.hdf5"
    else:
        dp += f"_subdiv_sliced_ws{ws}.hdf5"
    return dp
 

sim = "p21c"
ws = 0.0
n = 8
z=15
unenc_fname = get_path(sim, ws, False, False)
enc_fname = get_path(sim, ws, True, False)
norm_enc_fname = get_path(sim, ws, True, True)
fig_dir = f"../figures/data_figures/{sim}_ws{ws}"

start, end = 2,6
ntrvl=4
n = np.arange(start*ntrvl,end*ntrvl,ntrvl)
z = np.linspace(0, 29, 6, dtype=int)

if not os.path.exists(fig_dir): os.mkdir(fig_dir)



In [15]:

with h5py.File(enc_fname, 'r') as f:
    print(f['lightcones/brightness_temp'].shape)
    print(f['lightcone_params/physparams'].shape)
    sample = f['lightcones/brightness_temp'][n]
    labels = f['lightcone_params/physparams'][n]

sample = sample[:,z]
print(sample.shape)
print(labels.shape)

(2960, 30, 4, 32, 32)
(2960, 3)
(4, 6, 4, 32, 32)
(4, 3)


In [16]:
%matplotlib inline

for c in range(4):
    rowdict = {f"lc {ni} mdpt {labels[ni,0]:.1f}":sample[ni,:,c,:,:] for ni in range(len(n))}
    plot_image_rows(rowdict, fname=f"{fig_dir}/latent_channel_{c}_{sim}_ws{ws}.jpg", title=f"{sim} ws={ws} encoded channel {c}")

In [17]:
with h5py.File(norm_enc_fname, 'r') as f:
    print(f['lightcones/brightness_temp'].shape)
    print(f['lightcone_params/physparams'].shape)
    sample = f['lightcones/brightness_temp'][n]
    labels = f['lightcone_params/physparams'][n]

sample = sample[:,z]
print(sample.shape)
print(labels.shape)

(2960, 30, 4, 32, 32)
(2960, 3)
(4, 6, 4, 32, 32)
(4, 3)


In [18]:
%matplotlib inline

for c in range(4):
    rowdict = {f"lc {ni} mdpt {labels[ni,0]:.1f}":sample[ni,:,c,:,:] for ni in range(len(n))}
    plot_image_rows(rowdict, fname=f"{fig_dir}/prenorm_latent_channel_{c}_{sim}_ws{ws}.jpg", title=f"{sim} ws={ws} prenorm encoded channel {c}")

In [19]:

with h5py.File(unenc_fname, 'r') as f:
    print(f['lightcones/brightness_temp'].shape)
    input = f['lightcones/brightness_temp'][n]
    labels = f['lightcone_params/physparams'][n]
input = input[:,z]
print(input.shape)

(2960, 30, 256, 256)
(4, 6, 256, 256)


In [20]:
%matplotlib inline

rowdict = {f"n={ni} (Mdpt {labels[ni,0]:.1f})":input[ni,:,:,:] for ni in range(len(n))}

plot_image_rows(rowdict, fname=f"{fig_dir}/unencoded_bT_{sim}_ws{ws}.jpg", title=f"{sim} ws={ws} unencoded bT")


In [21]:
print(rowdict.keys())

dict_keys(['n=0 (Mdpt 9.2)', 'n=1 (Mdpt 7.9)', 'n=2 (Mdpt 6.8)', 'n=3 (Mdpt 7.4)'])
